# Постановка задачи и выяснение общей картины

In [27]:
import pandas as pd
df_new = pd.read_csv('ga_sessions.csv', low_memory=False)

GA Sessions (ga_sessions.pkl)

Одна строка = один визит на сайт.
Описание атрибутов:
session_id — ID визита;
client_id — ID посетителя;
visit_date — дата визита;
visit_time — время визита;
visit_number — порядковый номер визита клиента;
utm_source — канал привлечения;
utm_medium — тип привлечения;
utm_campaign — рекламная кампания;
utm_keyword — ключевое слово;
device_category — тип устройства;
device_os — ОС устройства;
device_brand — марка устройства;
device_model — модель устройства;
device_screen_resolution — разрешение экрана;
device_brand — марка устройства;
device_model — модель устройства;
device_screen_resolution — разрешение экрана;
device_browser — браузер;
geo_country — страна;
geo_city — город.

In [8]:
def print_fullycomplete_and_percent(df):
    print('количество полностью заполненных объектов ',len(df.dropna()))
    print('процент полностью заполненных объектов из всей выборки ',round(len(df.dropna()) / len(df) * 100, 2 ))

# Проведим разведочный анализ данных
# Проведим базовую чистку (дубликаты, пустые значения, типизация данных, ненужные атрибуты).

In [28]:
# копируем новый датасет
df_new2 = df_new.copy() 

In [29]:
# отбрасываем признаки с незначимыми аттрибутами (30% данных не заполнены): device_model, utm_keyword, device_os
df_new2 = df_new2.drop('device_model', axis = 1)
df_new2 = df_new2.drop('utm_keyword', axis = 1)
df_new2 = df_new2.drop('device_os', axis = 1)

In [33]:
# итого получаем рост заполненности данными 66%:
print_fullycomplete_and_percent(df_new2)

количество полностью заполненных объектов  1230859
процент полностью заполненных объектов из всей выборки  66.17


In [37]:
# итого получаем, что пустых данных нет
missing_values_percent = ( (df_new2.isna().sum() / len(df_new2))* 100 ).sort_values(ascending=False)
missing_values_percent 

session_id                  0.0
client_id                   0.0
visit_number                0.0
utm_source                  0.0
utm_medium                  0.0
utm_campaign                0.0
utm_adcontent               0.0
device_category             0.0
device_brand                0.0
device_screen_resolution    0.0
device_browser              0.0
geo_country                 0.0
geo_city                    0.0
visit_date_time             0.0
dtype: float64

In [31]:
# заполняем device_brand и utm_source самым  (это не потит картину по ним)
df_new2.device_brand = df_new2.device_brand.fillna((df_new2.device_brand.mode())[0])
df_new2.utm_source = df_new2.utm_source.fillna((df_new2.utm_source.mode())[0])

In [32]:
# а по рекламным компаниям заполняем поле OTHER
df_new2.utm_campaign = df_new2.utm_campaign.fillna('other')
df_new2.utm_adcontent = df_new2.utm_adcontent.fillna('other')

In [50]:
# теперь нет пропусков в данных

In [33]:
# преобразуем отдельно заданные дату и время в поле с типом Дата-Время
df_new2['visit_date_time'] = pd.to_datetime(df_new2['visit_date'] + ' ' + df_new2['visit_time'])

In [34]:
# удаляем лишние столбцы visit_date visit_time
df_new2 = df_new2.drop('visit_date', axis = 1)
df_new2 = df_new2.drop('visit_time', axis = 1)

In [35]:
# visit_numer лучше чтобы был Int
df_new2.visit_number = df_new2.visit_number.astype(int)

In [1]:
def convert_int(x):
    if len(x) == 0:
        return 0
    return int(x.split('.')[0])
 
def convert_float(x):
    if len(x) == 0:
        return np.nan
    return float(x)


df_hits = pd.read_csv('ga_hits.csv', low_memory=False, usecols= ['session_id','hit_date', 'hit_time', 'hit_number', 'hit_type','hit_referer','hit_page_path','event_category','event_action','event_label', 'event_value'], converters={'hit_time' : convert_int ,'event_value' : convert_int})

Описание второго дата-сета: 

GA Hits (ga_hits.pkl)
Одна строка = одно событие в рамках одного визита на сайт.
Описание атрибутов:
session_id — ID визита;
hit_date — дата события;
hit_time — время события;
hit_number — порядковый номер события в рамках сессии;
hit_type — тип события;
hit_referer — источник события;
hit_page_path — страница события;
event_category — тип действия;
event_action — действие;
event_label — тег действия;
event_value — значение результата действия.

In [4]:
df_hits2 = df_hits.copy()

In [21]:
#меняем тип столбца даты на дату время
df_hits2['hit_date'] = pd.to_datetime(df_hits2['hit_date'])

In [15]:
#заполняем Nan hit_referer event_label значениями other
df_hits2.hit_referer = df_hits2.hit_referer.fillna('other')
df_hits2.event_label = df_hits2.event_label.fillna('other')

In [25]:
# удаляем лишние столбцы visit_date visit_time
df_hits2 = df_hits2.drop('hit_time', axis = 1)

Выполняем задание согласно моей специализации Data Analyst:

Проведите проверку следующих гипотез:

1) Органический трафик не отличается от платного с точки зрения CR (Conversion Rate) в целевые события.

Целевое действие — события типа «Оставить заявку» и «Заказать звонок» 
(ga_hits.event_action in ['sub_car_claim_click', 'sub_car_claim_submit_click', 'sub_open_dialog_click', sub_custom_question_submit_click', 'sub_call_number_click', 'sub_callback_submit_click', 'sub_submit_success', 'sub_car_request_submit_click']).

CR (Conversion Rate) — показатель конверсии из визита (уникальный 
session_id) в любое целевое действие в рамках одного визита (в случае 
наличия >1 целевого действия — считать все как одно).


2) Трафик с мобильных устройств не отличается от трафика с десктопных устройств с точки зрения CR (Conversion Rate) в целевые события.

Органический трафик — все визиты с ga_sessions.utm_medium in ('organic', 
'referral', '(none)').

Платный трафик — весь неорганический трафик.

Информация про марку и модель авто — содержится 

в ga_hits.hit_page_path.
Реклама в социальных сетях — все визиты с ga_sessions.utm_source in 
('QxAxdyPLuQMEcrdZWdWb', 'MvfHsxITijuriZxsqZqt', 'ISrKoXQCxqqYvAZICvjs', 
IZEXUFLARCUMynmHNBGo', 'PlbkrSYoHuZBWfYjYnfw', 
'gVRrcxiDQubJiljoTbGm').

3) Трафик из городов присутствия (Москва и область, Санкт-Петербург) не отличается от трафика из иных регионов с точки зрения CR (Conversion Rate) в целевые события.


Ответьте на вопросы продуктовой команды:

1) Из каких источников (кампаний, устройств, локаций) к нам идёт самый целевой трафик (и с точки зрения объёма трафика, и с точки зрения CR)?

2) Какие авто пользуются наибольшим спросом? У каких авто самый лучший показатель CR (Conversion Rate) в целевые события?

3) Стоит ли нам увеличивать своё присутствие в соцсетях и давать там больше рекламы?


Объединяем дата-сет с событиями (когда кто подключался) и дата-сет с дополнительной инфрмацией о сессиях

In [13]:
df1 = pd.merge(df_hits2, df_new2, left_on='session_id', right_on='session_id')

Собираем в список все целевые действия и создаем новый признак равный целевому действию и не целевому действию

In [3]:
target_event_actions = [
    'sub_car_claim_submit_click', 
    'sub_submit_success', 
    'sub_car_claim_click', 
    'sub_open_dialog_click', 
    'sub_call_number_click',
    'sub_car_request_submit_click']
df1['event_action_taget_notarget'] = df1['event_action'].apply(lambda x : 'taget' if x in target_event_actions else 'notarget')


Собираем в список все типы органического траффика и создаем новый признак равный типу траффика

In [4]:
utm_medium_type_organic_not_organic = [
    '(none)', 
    'organic', 
    'referral']
df1['utm_medium_type_organic_not_organic'] = df1['utm_medium'].apply(lambda x : 'organic' if x in utm_medium_type_organic_not_organic else 'not_organic')


Собираем в список все типы устройства и создаем новый признак равный типу устройства

In [5]:
device_category_type = ['mobile', 'tablet']
df1['device_category_type'] = df1['device_category'].apply(lambda x : 'mobile' if x in device_category_type else 'desktop')


Собираем в список все города присутсвия и создаем новый признак равный городу присуствия

In [22]:
geo_city_presence = ['Saint Petersburg',
                    'Moscow',
                    'Podolsk',
                    'Korolyov',
                    'Kolomna',
                    'Naro-Fominsk',
                    'Dzerzhinsky',
                    'Krasnogorsk',
                    'Balashikha',
                    'Dmitrov',
                    'Sergiyev Posad',
                    'Odintsovo',
                    'Lobnya',
                    'Yegoryevsk',
                    'Kotelniki',
                    'Mytishchi',
                    'Zheleznodorozhny',
                    'Serpukhov',
                    'Shchyolkovo',
                    'Vidnoye',
                    'Khimki',
                    'Golitsyno',
                    'Ramenskoye',
                    'Dubna',
                    'Likino-Dulyovo',
                    'Ivanteyevka',
                    'LesnoyGorodok',
                    'Istra',
                    'Losino-Petrovsky',
                    'SaintPetersburg',
                    'Elektrogorsk',
                    'Protvino',
                    'Klimovsk',
                    'Tomilino',
                    'Petrovo-Dalneye',
                    'Pushkino',
                    'Orekhovo-Zuyevo',
                    'Voskresensk',
                    'Fryazino',
                    'Kraskovo',
                    'Khotkovo',
                    'Kubinka',
                    'Chernogolovka',
                    'Dolgoprudny']
df1['geo_city_presence'] = df1['geo_city'].apply(lambda x : 'city_presence' if x in geo_city_presence else 'other_city')


Создаем функцияю со списком моделей автомобиля и создаем новый признак - модель автомобиля 

In [9]:
def find_auto(str):
    car_models = ['/skoda', 
                  '/renault', 
                  '/volkswagen',
                  '/toyota', 
                  '/kia', 
                  '/nissan',
                  '/toyota',
                  '/volvo',
                  '/fiat',
                  '/mercedes-benz',
                  '/bmw',
                  '/volkswagen',
                  '/audi',
                  '/peugeot',
                  '/porsche',
                  '/haval',
                  '/lada-vaz',
                  '/mini',
                  '/lexus',
                  '/land-rover']
    found_car_model = 'model_unknown'
    for car_model in car_models:
        if car_model in str: 
            found_car_model = car_model
    return found_car_model
df1['car_type'] = df1['hit_page_path'].apply(lambda x : find_auto(x))


Строим сводную таблицу с группировкой по типу траффика (органика / неорганика) и типу конверсии - в целефвое действие и нецелевое действие

In [25]:
df1.groupby(['utm_medium_type_organic_not_organic','event_action_taget_notarget'])['session_id'].count()


utm_medium_type_organic_not_organic  event_action_taget_notarget
not_organic                          notarget                       9808238
                                     taget                            59414
organic                              notarget                       5777695
                                     taget                            39872
Name: session_id, dtype: int64

Нулевая гипотеза : кол-во конверсий среди органического трафика такое же, как и при не-органическом
Альтенативная гипотеза: кол-во конверсий выше при не-органическом траффике
Используем биномиальный критерий проверки гипотезы

Создаем функцию статистического Биноминального критерия значимости

In [2]:
def p_bin(m1,n1,m2,n2,level_conf=0.05): 
    # на вход ф-ии подается кол-во успехов варианта А и общее число вариантов m1,n1 
    #и число успехов варанта Б и общее число вариантов m2, n2, 
    #а также уровень значимости - опционально (0.05 по умолчанию)
    from scipy import stats
    T = (m1/n1 - m2/n2)/((m1+m2)/(n1+n2)*(1 - (m1+m2)/(n1+n2))*(1/n1+1/n2))**0.5 #формула биномиального критерия вер-сти
    P = 1 - stats.norm.cdf(T)  # вычисляем p-значение
    # критичная область у нас со знаком БОЛЬШЕ, потому что задачался вопрос в альтернативной что ВАРИАНТ А был более значимым
    print(f"Stats {T}, p-value {P}")
    if (P < level_conf):
        print(f"На уровне значимости {level_conf} нулевая гипотеза отвергается в пользу альтернативной: Вариаент А более значим (значимая разница Варианта А перед вариантом Б есть)")
    else:
        print(f"На уровне значимости {level_conf} нулевая гипотеза принимается (нет значимой разницы между вариантом А и Б)")
#p_bin(46,1100,30,1000,0.1) Пример ф-иии и ответа:
# На уровне значимости 0.1 нулевая гипотеза отвергается в пользу альтернативной: Вариаент А более значим (значимая разница Варианта А перед вариантом Б есть)

In [3]:
p_bin(59414, 9808238+59414,39872,5777695+39872)

Stats -20.084816152154797, p-value 1.0
На уровне значимости 0.05 нулевая гипотеза принимается (нет значимой разницы между вариантом А и Б)


Строим сводную таблицу с группировкой по типу устройства  и типу конверсии - в целефвое действие и нецелевое действие

In [26]:
df1.groupby(['device_category_type','event_action_taget_notarget'])['session_id'].count()


device_category_type  event_action_taget_notarget
desktop               notarget                        3921408
                      taget                             23846
mobile                notarget                       11664525
                      taget                             75440
Name: session_id, dtype: int64

Нулевая гипотеза : кол-во конверсий среди мобильного траффика такое же, как и при траффике с ПК
Альтенативная гипотеза: кол-во конверсий выше при мобильном траффике
Используем биномиальный критерий проверки гипотезы

In [4]:
p_bin(75440, 11664525+75440,23846,3921408+23846)

Stats 8.270194225881966, p-value 1.1102230246251565e-16
На уровне значимости 0.05 нулевая гипотеза отвергается в пользу альтернативной: Вариаент А более значим (значимая разница Варианта А перед вариантом Б есть)


Строим сводную таблицу с группировкой по городам присуствия и нет а также и типу конверсии - в целефвое действие и нецелевое действие

In [27]:
df1.groupby(['geo_city_presence', 'event_action_taget_notarget'])['session_id'].count()


geo_city_presence  event_action_taget_notarget
city_presence      notarget                       9847695
                   taget                            67130
other_city         notarget                       5738238
                   taget                            32156
Name: session_id, dtype: int64

Нулевая гипотеза : кол-во конверсий среди трафика из городов присутсвия такое же, как из других городов
Альтенативная гипотеза: кол-во конверсий выше из городов присутсвия
Используем биномиальный критерий проверки гипотезы
Итого, Нулевая гипотеза отвергается в пользу альтернативной - больше конверсий из городов присутсвия.

In [5]:
p_bin(67130, 9847695+67130,32156,5738238+32156)

Stats 28.851484895594773, p-value 0.0
На уровне значимости 0.05 нулевая гипотеза отвергается в пользу альтернативной: Вариаент А более значим (значимая разница Варианта А перед вариантом Б есть)


Строим сводную таблицу с группировкой по типу конверсии - в целевое действие и нецелевое действие, типу траффика и типу устройств, а также городам присуствия. 

In [28]:
df1.groupby(['event_action_taget_notarget', 'utm_medium_type_organic_not_organic', 'device_category_type' ,'geo_city_presence'])['session_id'].count()


event_action_taget_notarget  utm_medium_type_organic_not_organic  device_category_type  geo_city_presence
notarget                     not_organic                          desktop               city_presence        1203764
                                                                                        other_city            349756
                                                                  mobile                city_presence        4864114
                                                                                        other_city           3390604
                             organic                              desktop               city_presence        1517897
                                                                                        other_city            849991
                                                                  mobile                city_presence        2261920
                                                                           

С точки зрения объема траффика к нам идет наибольший неорганический траффик с мобильных устройств из городов присуствия
С точки зрения конверсии, к нам идет неорганический траффик с мобильных устройств из городов присутсвия

Строим сводную таблицу с группировкой по типу конверсии - в целевое действие и нецелевое действие, а также модели автомобиля

In [32]:
df1.groupby(['event_action_taget_notarget', 'car_type'])['session_id'].count()


event_action_taget_notarget  car_type      
notarget                     /audi                62724
                             /bmw                193573
                             /fiat                   48
                             /haval               51264
                             /kia                237353
                             /lada-vaz           399928
                             /land-rover          24879
                             /lexus               61586
                             /mercedes-benz      471686
                             /mini                34724
                             /nissan             364201
                             /peugeot             27247
                             /porsche             63350
                             /renault            147942
                             /skoda              736995
                             /toyota             162238
                             /volkswagen         411587
    

Самые популярные модели с точки зрения наибольшего спроса (ТОП5) это skoda, lada-vaz, volkswagen, kia, mercedes-benz

# ОТВЕТЫ на Гипотезы и Вопросы:

1) Органический трафик не отличается от платного с точки зрения CR (Conversion Rate) в целевые события.
Нулевая гипотеза : кол-во конверсий среди органического трафика такое же, как и при не-органическом 
    Альтенативная гипотеза: кол-во конверсий выше при не-органическом траффике Используем биномиальный критерий проверки гипотезы
На уровне значимости 0.05 нулевая гипотеза ПОДТВЕРЖДАЕТСЯ (нет значимой разницы между вариантом А и Б), 
т.е. Органический трафик не отличается от платного с точки зрения CR (Conversion Rate) в целевые события.

2) Трафик с мобильных устройств не отличается от трафика с десктопных устройств с точки зрения CR (Conversion Rate) в целевые события.
Нулевая гипотеза : кол-во конверсий среди мобильного траффика такое же, как и при траффике с ПК 
    Альтенативная гипотеза: кол-во конверсий выше при мобильном траффике Используем биномиальный критерий проверки гипотезы
На уровне значимости 0.05 нулевая гипотеза ОТВЕРГАЕТСЯ в пользу альтернативной: Вариаент А более значим (значимая разница Варианта А перед вариантом Б есть)
т.е. трафик с мобильных устройств ОТЛИЧАЕТСЯ от трафика с десктопных устройств с точки зрения CR (Conversion Rate) в целевые события.    
        
3) Трафик из городов присутствия (Москва и область, Санкт-Петербург) не отличается от трафика из иных регионов с точки зрения CR (Conversion Rate) в целевые события.
Нулевая гипотеза : кол-во конверсий среди трафика из городов присутствия такое же, как из других городов
    Альтенативная гипотеза: кол-во конверсий выше из городов присутствия
Используем биномиальный критерий проверки гипотезы
На уровне значимости 0.05 нулевая гипотеза ОТВЕРГАЕТСЯ в пользу альтернативной: Вариаент А более значим (значимая разница Варианта А перед вариантом Б есть)
т.е. трафик из городов присутствия (Москва и область, Санкт-Петербург) ОТЛИЧАЕТСЯ от трафика из иных регионов с точки зрения CR (Conversion Rate) в целевые события.
    
Ответьте на вопросы продуктовой команды:

1) Из каких источников (кампаний, устройств, локаций) к нам идёт самый целевой трафик (и с точки зрения объёма трафика, и с точки зрения CR)?
С точки зрения объема траффика к нам идет наибольший неорганический траффик с мобильных устройств из городов присуствия
С точки зрения конверсии, к нам идет неорганический траффик с мобильных устройств из городов присутсвия

2) Какие авто пользуются наибольшим спросом? У каких авто самый лучший показатель CR (Conversion Rate) в целевые события?
Самые популярные модели с точки зрения наибольшего спроса (ТОП5) это skoda, lada-vaz, volkswagen, kia, mercedes-benz

3) Стоит ли нам увеличивать своё присутствие в соцсетях и давать там больше рекламы?
В связи с тем, что неорганический траффик НЕ имеет решающее значение для повышения конверсии, 
НЕ следует увеличить свое присутвие в социальных сетях, давать больше рекламы. 
При этом следует обратить внимание, что наибольшую конерсию мы получаем через мобильные устройства и 
из городов присутсвия: следует сконцентрироваться на мобильном сегменте, мобильном приложении и на 
городах присутвия, слеует найти самые популярные соц-сети через мобильные устройства и концентрироваться на 
тех моделях, что наиболее интересны аудитории (Шкода, Лада-Ваз, Фолькцваген, Киа, Мерседес) 
также мобильные интеграции.